In [12]:
from copy import copy
import numpy as numpy

In [38]:
def gamess_guga(file_name,threshold=0.0,**kwargs):
  '''Reads GAMESS-US GUGA output.

  **Parameters:**

    file_name: str, file descriptor
      Specifies the filename for the input file.
      fname can also be used with a file descriptor instad of a filename.
    threshold : float, optional
      Specifies a read threshold for the CI coefficients.

  **Returns:**

    ci : list of CIinfo class instances
      See :ref:`Central Variables` for details.
  '''
  display('\nReading data of GUGA calculation from GAMESS-US...')

  ci = [[]]
  general_information = {'fileinfo': file_name,
                         'read_threshold': threshold}


  fname = open(file_name, 'r')


  from io import TextIOWrapper
  if isinstance(fname, TextIOWrapper):
    flines = fname.readlines()

  STATE_ZONE=False
  state_energies = []
  state_CSFs = []
  state_coeffs = []
  state_OCCs = []

  NUMCI = 0
  NFZC = []
  NMCC = []
  NDOC = []
  NAOS = []
  NBOS = []
  NALP = []
  NVAL = []
  NEXT = []
  NFZV = []
  nel = []
  spin = []

  for i,line in enumerate(flines):
    if STATE_ZONE:
      thisline = line.split()
      if " STATE #" in line:
        state_energies[-1].append(float(thisline[-1]))
        temp_state_CSFs = []
        temp_state_OCCs = []
        temp_state_coeffs = []
        continue
      if len(thisline) == 3 and abs(float(thisline[1])) >= threshold:
        temp_state_CSFs.append(int(thisline[0]))
        temp_state_coeffs.append(float(thisline[1]))
        temp = [int(x) for x in thisline[2]]
        temp_state_OCCs.append(temp)
        continue
      if " THE " in line:
        state_CSFs[-1].append(temp_state_CSFs)
        state_coeffs[-1].append(temp_state_coeffs)
        state_OCCs[-1].append(temp_state_OCCs)
        continue
      if "STEP CPU TIME" in line:
        STATE_ZONE = False
        continue
    if "CI CALCULATION" in line:
      NUMCI += 1
      ci.append([])
      state_energies.append([])
      state_CSFs.append([])
      state_coeffs.append([])
      state_OCCs.append([])
      NFZC.append(int(flines[i+14].split()[1]))
      NDOC.append(int(flines[i+14].split()[3]))
      NEXT.append(int(flines[i+14].split()[5]))
      NMCC.append(int(flines[i+15].split()[1]))
      NAOS.append(int(flines[i+15].split()[3]))
      NFZV.append(int(flines[i+15].split()[5]))
      NBOS.append(int(flines[i+16].split()[1]))
      NALP.append(int(flines[i+17].split()[1]))
      NVAL.append(int(flines[i+18].split()[1]))
      nel.append( 2*(NFZC[-1]+NDOC[-1]+NMCC[-1]) + 1*(NAOS[-1]+NBOS[-1]+NALP[-1]))
      spin.append( 1 + NALP[-1] )
      continue

    if " NUMBER OF STATES REQUESTED =" in line:
      STATE_ZONE=True
      numstates = int(line.split()[-1])
      continue

  fname.close()

  NCORE = [sum(i) for i in zip(*[NFZC,NMCC])]
  NACT = [sum(i) for i in zip(*[NDOC,NAOS,NBOS,NALP,NVAL,NEXT])]

  CSFs_l = []
  for stg_CSFs in state_CSFs:
    CSFs = set()
    for st_CSFs in stg_CSFs:
      for c in st_CSFs:
        CSFs.add(c)
    CSFs_l.append(sorted(list(CSFs)))

  CSFs = [{}]

  fname = open(file_name, 'r')

  if isinstance(fname, TextIOWrapper):
    flines = fname.readlines()

  CI_index = 0
  SKIP = 0

  for i,line in enumerate(flines):
    if "CI CALCULATION" in line:
      SKIP = 0
    if SKIP:
      continue
    if  " CASE VECTOR =" in line:
      if int(line.split()[3]) == CSFs_l[CI_index][0]:
        CSFs[CI_index][CSFs_l[CI_index][0]] = [int(x) for x in flines[i+1].split()[0]]
        CSFs_l[CI_index].pop(0)
        if len(CSFs_l[CI_index])==0:
          CI_index += 1
          CSFs.append({})
          SKIP = 1
          continue

  CSFs.pop(-1)

  fname.close()

  spinz = []
  for s in spin:
    spinz.append([s-1-2*x for x in range(s)])
    
  state_num = 0
  for CI_i in range(NUMCI):
    for i in range(len(state_energies[CI_i])):
      for ms in spinz[CI_i]:
        ci[CI_i].append({})
        ci[CI_i][-1]["info"] = copy(general_information)
        ci[CI_i][-1]["info"]["state"] = state_num
        state_num += 1
        ci[CI_i][-1]["info"]["energy"] = state_energies[CI_i][i]
        ci[CI_i][-1]["info"]["spin"] = spin[CI_i]
        ci[CI_i][-1]["info"]["ms"] = ms/2
        ci[CI_i][-1]["info"]["nel"] = nel[CI_i]
        ci[CI_i][-1]["info"]["occ_info"] = [NCORE[CI_i],NACT[CI_i],NFZV[CI_i]]
        ci[CI_i][-1]["coeffs"] = []
        ci[CI_i][-1]["occ"] = []
        ci[CI_i][-1]["cv"] = []
        for j,csfi in enumerate(state_CSFs[CI_i][i]):
          CGCs, OCCs = CV_to_dets(CSFs[CI_i][csfi], NCORE[CI_i], NDOC[CI_i], NALP[CI_i], NVAL[CI_i], ms)
          for k in range(len(CGCs)):
            ci[CI_i][-1]["coeffs"].append(state_coeffs[CI_i][i][j]*CGCs[k])
            ci[CI_i][-1]["occ"].append(OCCs[k])
            ci[CI_i][-1]["cv"].append(CSFs[CI_i][csfi])
        ci[CI_i][-1]["coeffs"] = numpy.array(ci[CI_i][-1]["coeffs"])
        ci[CI_i][-1]["occ"] = numpy.array(ci[CI_i][-1]["occ"])

  #--- Calculating norm of CI states
  display('\nIn total, %d states have been read.' % sum([len(i) for i in ci]))
  display('Norm of the states:')
  for i in range(len(ci)):
    for j in range(len(ci[i])):
      norm = sum([x*x for x in ci[i][j]['coeffs']])
      display('State %s (%s,%s):Norm = %0.8f (%d Coefficients)' %
              (ci[i][j]["info"]['state'],ci[i][j]['info']['spin'],ci[i][j]['info']['ms'],
               norm,len(ci[i][j]['coeffs'])))
  display('')

  ci_new = []
  for i in ci:
    ci_new.extend(i)

  return ci_new



In [2]:
def CV_to_dets(CV, NCORE, NDOC, NALP, NVAL, MS):
    """
    Turn Case Vector into Determinants and Coefficients
    Not coded for NAOS or NBOS, can be adjusted
    GAMESS Case Vector ascends through unoccupied, descends through doubly occupied, the descends through singly occupied
    1 unoccupied
    2 single, raises total spin
    3 single, lowers total spin
    4 is doubly occupied ( necessarily singlet, see Szabo Ex. 2.38 )
    """

    NPHASE = 1
    IB = 0
    NOPEN = 0
    s = [] # Says if electron raises or lowers total spin (left to right)
    IORDER = [x for x in range(NCORE+NDOC+NALP+1, NCORE+NDOC+NALP+NVAL+1)] \
           + [x for x in range(NCORE+NDOC, NCORE, -1)] \
           + [x for x in range(NCORE+NDOC+NALP, NCORE+NDOC, -1)]

    for c in CV:
        if c == 1:
            continue
        if c == 2:
            IB += 1
            NOPEN += 1
            s.append(1)
            continue

        if c == 3:
            IB -= 1
            NOPEN += 1
            s.append(-1)
            continue

        if c == 4:
            if IB%2:
                NPHASE *= -1 # Enforce Antisymmetry under Exchange?
            continue

    s_z, CGCs, iCGCs = SPNFNC(NOPEN, s, MS) # Have to go through all Ms when relativistic

    CGCs = [NPHASE*x for x in CGCs]
    iCGCs = [NPHASE*x for x in iCGCs]
    OCCs = []

    for i,sz in enumerate(s_z):
        DET = MAKDET(CV, IORDER, sz)
        occupation = numpy.zeros((NDOC+NALP+NVAL,2),dtype=numpy.intc)
        for orb in DET:
            if orb > 0:
                occupation[orb-NCORE-1,0] = 1
            elif orb < 0:
                occupation[abs(orb)-NCORE-1,1] = 1
        OCCs.append(occupation)

    return CGCs, OCCs


In [3]:
def SPNFNC(NOPEN, s=[0], Ms=0):
    """
    Generates spin functions
    """

    if NOPEN == 0:
        return [0], [1], [1]

    s_z = [[1],[-1]]
    for i in range(NOPEN-1):
        s_z = [x+[1] for x in s_z] + [x+[-1] for x in s_z] # All permutations of spin_z for opens (off by factor of 2)

    S_z = []
    for sz in s_z:
        S_z.append([sum(sz[:1+i]) for i in range(NOPEN)]) # All total S_z as one travels along the spins

    pop_ind = 0
    while pop_ind < len(s_z):
        if S_z[pop_ind][-1] == Ms: # Exclude determinants with non-allowed total spin
            pop_ind += 1
        else:
            S_z.pop(pop_ind)
            s_z.pop(pop_ind)

    S = [sum(s[:1+i]) for i in range(NOPEN)] # Total Spin tracked from left to right

    pop_ind = 0
    while pop_ind < len(s_z):
        for j in range(NOPEN):
            if abs(S_z[pop_ind][j]) > S[j]: # Exclude determinants with non-allowed total spin couplings
                S_z.pop(pop_ind)
                s_z.pop(pop_ind)
                pop_ind -= 1
                break
        pop_ind += 1

    #   "Calculate the Clebsh-Gordon Coefficients"

    CGCs = []
    iCGCs = []

    for i in range(len(s_z)):
        COEF = 1
        for j in range(NOPEN):
            if s[j] == 1:
                if s_z[i][j] == 1:
                    COEF *= (S[j]+S_z[i][j])
                else:
                    COEF *= (S[j]-S_z[i][j])
                COEF /= 2*S[j]
            else:
                if s_z[i][j] == 1:
                    COEF *= (S[j]-S_z[i][j]+2)
                    if S[j]%2 == 0:
                        COEF *= -1
                else:
                    COEF *= (S[j]+S_z[i][j]+2)
                    if S[j]%2 == 1:
                        COEF *= -1
                COEF /= (2*S[j] + 4)
        if COEF == 0:
            continue
        CGCs.append(COEF/abs(COEF)**(1/2))
        iCGCs.append(1/COEF)

    return s_z, CGCs, iCGCs


In [4]:
def MAKDET(CV,IORDER,sz):
    DET = []
    iSO = 0
    for i,cv in enumerate(CV):
        if cv == 1:
            continue
        elif cv == 4:
            DET.append(IORDER[i])
            DET.append(-IORDER[i])
        else:
            if sz[iSO] == 1:
                DET.append(IORDER[i])
            else:
                DET.append(-IORDER[i])
            iSO += 1
    return DET


In [39]:
ci = gamess_guga("socisd3.out")

'\nReading data of GUGA calculation from GAMESS-US...'

'\nIn total, 180 states have been read.'

'Norm of the states:'

'State 0 (1,0.0):Norm = 0.93857370 (45 Coefficients)'

'State 1 (1,0.0):Norm = 0.96505103 (46 Coefficients)'

...

'State 176 (5,1.0):Norm = 0.91747172 (228 Coefficients)'

'State 177 (5,0.0):Norm = 0.91747172 (320 Coefficients)'

'State 178 (5,-1.0):Norm = 0.91747172 (228 Coefficients)'

'State 179 (5,-2.0):Norm = 0.91747172 (74 Coefficients)'

''

In [40]:
for st in ci:
    ben_can_call_this_whatever = {}
    for oc in st['occ']:
        k = "".join(str(y) for y in [x[0]+2*x[1] for x in oc])
        ben_can_call_this_whatever[k] = 0 # Makes a dictionary k for all significant determinants
    for i,oc in enumerate(st['occ']):
        k = "".join(str(y) for y in [x[0]+2*x[1] for x in oc])
        ben_can_call_this_whatever[k] += st['coeffs'][i]
    st['dets'] = ben_can_call_this_whatever

In [41]:
ci[0]['coeffs']

array([ 0.916627  , -0.0752298 ,  0.0752298 , -0.03917089,  0.03917089,
       -0.067441  , -0.0752298 ,  0.0752298 , -0.03917089,  0.03917089,
        0.05359081, -0.0267954 , -0.0267954 , -0.0267954 , -0.0267954 ,
        0.05359081,  0.026151  , -0.026151  , -0.026151  ,  0.026151  ,
       -0.067441  ,  0.03602073, -0.03602073, -0.04357378,  0.02178689,
        0.02178689,  0.02178689,  0.02178689, -0.04357378, -0.0346615 ,
        0.0346615 ,  0.0346615 , -0.0346615 , -0.04357378,  0.02178689,
        0.02178689,  0.02178689,  0.02178689, -0.04357378, -0.0346615 ,
        0.0346615 ,  0.0346615 , -0.0346615 , -0.148465  , -0.059202  ])

In [42]:
ci[0]['occ']

array([[[1, 1],
        [1, 1],
        [1, 1],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]],

       [[1, 1],
        [1, 1],
        [1, 1],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]],

       [[1, 1],
        [1, 1],
        [1, 1],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]],

       ...,

       [[1, 1],
        [1, 1],
        [1, 1],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]],

       [[1, 1],
        [1, 1],
        [1, 1],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]],

       [[1, 1],
        [1, 1],
        [1, 1],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]]], dtype=int32)

In [43]:
ci[0]['cv']

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4],
 [1, 1, 1, 2, 1, 1, 1, 1, 1, 4, 4, 4, 3, 4, 4, 4, 4, 4],
 [1, 1, 1, 2, 1, 1, 1, 1, 1, 4, 4, 4, 3, 4, 4, 4, 4, 4],
 [1, 1, 1, 2, 1, 1, 1, 1, 1, 4, 4, 4, 4, 3, 4, 4, 4, 4],
 [1, 1, 1, 2, 1, 1, 1, 1, 1, 4, 4, 4, 4, 3, 4, 4, 4, 4],
 [1, 1, 1, 4, 1, 1, 1, 1, 1, 4, 4, 4, 1, 4, 4, 4, 4, 4],
 [1, 1, 2, 1, 1, 1, 1, 1, 1, 4, 4, 3, 4, 4, 4, 4, 4, 4],
 [1, 1, 2, 1, 1, 1, 1, 1, 1, 4, 4, 3, 4, 4, 4, 4, 4, 4],
 [1, 1, 2, 1, 1, 1, 1, 1, 1, 4, 4, 4, 4, 4, 3, 4, 4, 4],
 [1, 1, 2, 1, 1, 1, 1, 1, 1, 4, 4, 4, 4, 4, 3, 4, 4, 4],
 [1, 1, 2, 2, 1, 1, 1, 1, 1, 4, 4, 3, 3, 4, 4, 4, 4, 4],
 [1, 1, 2, 2, 1, 1, 1, 1, 1, 4, 4, 3, 3, 4, 4, 4, 4, 4],
 [1, 1, 2, 2, 1, 1, 1, 1, 1, 4, 4, 3, 3, 4, 4, 4, 4, 4],
 [1, 1, 2, 2, 1, 1, 1, 1, 1, 4, 4, 3, 3, 4, 4, 4, 4, 4],
 [1, 1, 2, 2, 1, 1, 1, 1, 1, 4, 4, 3, 3, 4, 4, 4, 4, 4],
 [1, 1, 2, 2, 1, 1, 1, 1, 1, 4, 4, 3, 3, 4, 4, 4, 4, 4],
 [1, 1, 2, 3, 1, 1, 1, 1, 1, 4, 4, 2, 3, 4, 4, 4, 4, 4],
 [1, 1, 2, 3, 1, 1, 1, 1, 1, 4,

In [44]:
ci[0]['dets']

{'333333333000000000': 0.916627,
 '333331333000200000': -0.07522979755721797,
 '333332333000100000': 0.07522979755721797,
 '333313333000200000': -0.039170887250609986,
 '333323333000100000': 0.039170887250609986,
 '333330333000300000': -0.067441,
 '333333133002000000': -0.07522979755721797,
 '333333233001000000': 0.07522979755721797,
 '333133333002000000': -0.039170887250609986,
 '333233333001000000': 0.039170887250609986,
 '333331133002200000': 0.05359080668671944,
 '333331233002100000': -0.0006444033433597209,
 '333331233001200000': -0.05294640334335972,
 '333332133002100000': -0.05294640334335972,
 '333332133001200000': -0.0006444033433597209,
 '333332233001100000': 0.05359080668671944,
 '333333033003000000': -0.067441,
 '333333313020000000': 0.03602072654042392,
 '333333323010000000': -0.03602072654042392,
 '333331313020200000': -0.04357377951627943,
 '333331323020100000': -0.012874610241860283,
 '333331323010200000': 0.05644838975813971,
 '333332313020100000': 0.05644838975813971,

In [45]:
norm = 0
for k in ci[0]['dets']:
    norm += ci[0]['dets'][k]**2
norm

0.9385737011089995

Same norm as state 0 using CSFs

3 is double

1 is alpha

2 is beta

0 is unoccupied